### Import libraries and custom functions

In [1]:
# libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

# custom functions
from CleanData import *
from MergeData import *
from AugmentData import *
from CorrectHeading import *
from PlotData import *

### Correct, augment, and combine data

In [3]:
windspeeds = ["30cms", "40cms", "60cms"]

for windspeed in windspeeds:
    # import data 
    trajec_df = pd.read_hdf(f"../ExperimentalData/{windspeed}/flight_trajectories_3d_HCS_odor_horizon_matched.h5")
    body_df = pd.read_hdf(f"../ExperimentalData/{windspeed}/body_orientations_HCS_odor_horizon_matched.h5")
    key_table = pd.read_hdf(f"../ExperimentalData/{windspeed}/body_trajec_matches.h5")

    # correct and synchronize data
    trajec_df = correct_for_wind(trajec_df)
    trajec_df = remove_irrelevant_trajectory_data(trajec_df)
    synced_trajectory = sync_time(trajec_df)
    body_df = remove_irrelevant_body_data(body_df)
    synced_body = sync_time(body_df)

    # merge all trajectory and body dfs 
    all_flies_body_and_trajectory = join_all_body_and_trajectory(synced_body, synced_trajectory, key_table)

    # only consider dfs at or longer than 5 time steps
    filtered_dfs = [df for df in all_flies_body_and_trajectory if len(df) > 12]

    # augment fly dfs with addition fields for neural network
    augmented_all_flies_body_and_trajectory = [augment_df(fly_body_and_trajectory) for fly_body_and_trajectory in filtered_dfs]

    # concatenate all dataframes vertically
    fly_data = pd.concat(augmented_all_flies_body_and_trajectory, ignore_index=True)
    fly_data.to_csv(f"../ExperimentalData/{windspeed}_corrected_fly_trajectories_with_body_orientations.csv",index=False)

In [5]:
# Read and concatenate all CSV files
file_paths = ["../ExperimentalData/30cms_corrected_fly_trajectories_with_body_orientations.csv",
              "../ExperimentalData/40cms_corrected_fly_trajectories_with_body_orientations.csv",
              "../ExperimentalData/60cms_corrected_fly_trajectories_with_body_orientations.csv"]
dataframes = [pd.read_csv(file) for file in file_paths]
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('../ExperimentalData/combined_corrected_fly_trajectories_with_body_orientations.csv', index=False)